In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_csv_files(filepath):
    df = pd.read_csv(filepath)
    return df

In [3]:
def convert_object_to_date(sample_date_text):
    return pd.to_datetime(sample_date_text, format='%m/%d/%Y')

In [4]:
jan_df  = read_csv_files('TableA.csv')
feb_df  = read_csv_files('TableB.csv')
mar_df  = read_csv_files('TableC.csv')
city_df = read_csv_files('TableD.csv')

In [5]:
jan_df.head()

,booking_id,customer_id,source,status,checkin,checkout,oyo_rooms,hotel_id,amount,discount,date
0,1,189314,4,3,2/4/2017,2/5/2017,1,252,3160,669.0,1/31/2017
1,2,46268,4,3,1/27/2017,1/28/2017,1,252,1893,481.0,1/27/2017
2,3,55271,3,2,1/25/2017,1/26/2017,1,252,2188,463.0,1/25/2017
3,4,170766,4,3,1/26/2017,1/27/2017,1,252,3054,646.0,1/26/2017
4,5,170766,4,3,1/26/2017,1/28/2017,1,252,6107,1293.0,1/25/2017


In [6]:
print feb_df.shape

(928, 11)


In [7]:
feb_df.head()

,booking_id,customer_id,source,status,checkin,checkout,oyo_rooms,hotel_id,amount,discount,date
0,1001,32661,4,2,2/19/2017,2/20/2017,1,252,2506,530.0,2/19/2017
1,1002,77901,1,2,2/17/2017,2/18/2017,1,252,2573,545.0,2/14/2017
2,1003,36635,4,2,2/16/2017,2/17/2017,1,252,2616,553.0,2/15/2017
3,1004,36635,4,2,2/14/2017,2/16/2017,1,252,5478,1159.0,2/11/2017
4,1005,193104,4,2,2/12/2017,2/14/2017,1,252,5502,1164.0,2/12/2017


In [8]:
mar_df.head()

,booking_id,customer_id,source,status,checkin,checkout,oyo_rooms,hotel_id,amount,discount,date
0,2001,189314,4,3,4/2/2017,4/3/2017,1,252,3215,680.0,3/26/2017
1,2002,77901,4,2,4/1/2017,4/2/2017,1,252,2675,567.0,3/31/2017
2,2003,199763,4,3,3/24/2017,3/25/2017,1,252,2652,561.0,3/24/2017
3,2004,162631,0,2,3/21/2017,3/22/2017,1,252,3366,740.0,3/20/2017
4,2005,119697,2,2,3/17/2017,3/18/2017,2,252,6399,1354.0,3/17/2017


In [9]:
print city_df.head()
print city_df.shape

   hotel_id     city
0         3  Gurgaon
1        13  Gurgaon
2        16  Gurgaon
3        21  Gurgaon
4        25    Delhi
(357, 2)


In [10]:
merged_feb_df = pd.merge(left=feb_df,right=city_df, left_on='hotel_id', right_on='hotel_id')
merged_jan_df = pd.merge(left=jan_df,right=city_df, left_on='hotel_id', right_on='hotel_id')
merged_mar_df = pd.merge(left=mar_df,right=city_df, left_on='hotel_id', right_on='hotel_id')

In [11]:
merged_jan_df[['date','checkin','checkout']] =  merged_jan_df[['date','checkin','checkout']] \
                                                .apply(convert_object_to_date)
merged_jan_df['room_nights'] = merged_jan_df['oyo_rooms']*\
                              ((merged_jan_df['checkout'] - merged_jan_df['checkin']).dt.days)
merged_jan_df['avg_price_per_room'] = merged_jan_df['amount'].astype(float)/merged_jan_df['room_nights']

In [12]:
merged_feb_df[['date','checkin','checkout']] =  merged_feb_df[['date','checkin','checkout']] \
                                                .apply(convert_object_to_date)
merged_feb_df['room_nights'] = merged_feb_df['oyo_rooms']*\
                              ((merged_feb_df['checkout'] - merged_feb_df['checkin']).dt.days)
merged_feb_df['avg_price_per_room'] = merged_feb_df['amount'].astype(float)/merged_feb_df['room_nights']

In [13]:
merged_mar_df[['date','checkin','checkout']] =  merged_mar_df[['date','checkin','checkout']] \
                                                .apply(convert_object_to_date)
merged_mar_df['room_nights'] = merged_mar_df['oyo_rooms']*\
                              ((merged_mar_df['checkout'] - merged_mar_df['checkin']).dt.days)
merged_mar_df['avg_price_per_room'] = merged_mar_df['amount'].astype(float)/merged_mar_df['room_nights']

In [14]:
merged_mar_df.count()

booking_id            961
customer_id           961
source                961
status                961
checkin               961
checkout              961
oyo_rooms             961
hotel_id              961
amount                961
discount              947
date                  961
city                  961
room_nights           961
avg_price_per_room    961
dtype: int64

In [15]:
jan_df = merged_jan_df.groupby(["customer_id"]).agg({"amount": ['sum','count'],"room_nights":'sum'}).reset_index()
jan_df.columns = [' '.join(col).strip() for col in jan_df.columns.values]
jan_df.rename(columns={'amount sum': 'total_amt_jan', 'amount count': 'no_bookings_jan',\
                       'room_nights sum':'total_room_nights_jan'}, inplace=True)
print jan_df.shape
jan_df.head()

(719, 4)


,customer_id,total_amt_jan,no_bookings_jan,total_room_nights_jan
0,220,3443,2,2
1,765,1949,1,1
2,1864,4288,2,2
3,2162,4159,1,1
4,2202,15918,2,5


In [16]:
feb_df = merged_feb_df.groupby(["customer_id"]).agg({"amount": ['sum','count'],"room_nights":'sum'}).reset_index()
feb_df.columns = [' '.join(col).strip() for col in feb_df.columns.values]
feb_df.rename(columns={'amount sum': 'total_amt_feb', 'amount count': 'no_bookings_feb',\
                       'room_nights sum':'total_room_nights_feb'}, inplace=True)
print feb_df.shape
feb_df.head()

(699, 4)


,customer_id,total_amt_feb,no_bookings_feb,total_room_nights_feb
0,211,1754,1,1
1,292,5719,1,1
2,1731,1819,1,1
3,1864,1494,1,1
4,2029,3119,1,1


In [17]:
mar_df = merged_mar_df.groupby(["customer_id"]).agg({"amount": ['sum','count'],"room_nights":'sum'}).reset_index()
mar_df.columns = [' '.join(col).strip() for col in mar_df.columns.values]
mar_df.rename(columns={'amount sum': 'total_amt_mar', 'amount count': 'no_bookings_mar',\
                       'room_nights sum':'total_room_nights_mar'}, inplace=True)
mar_df.head()

,customer_id,total_amt_mar,no_bookings_mar,total_room_nights_mar
0,220,1949,1,1
1,648,14815,1,4
2,714,2267,1,1
3,728,4677,1,2
4,834,3769,1,1


In [18]:
merged_df = pd.merge(pd.merge(jan_df, feb_df,on='customer_id',how='outer'),mar_df,on='customer_id',how='outer')
merged_df.head()

,customer_id,total_amt_jan,no_bookings_jan,total_room_nights_jan,total_amt_feb,no_bookings_feb,total_room_nights_feb,total_amt_mar,no_bookings_mar,total_room_nights_mar
0,220,3443.0,2.0,2.0,NaN,NaN,NaN,1949.0,1.0,1.0
1,765,1949.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1864,4288.0,2.0,2.0,1494.0,1.0,1.0,NaN,NaN,NaN
3,2162,4159.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2202,15918.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
merged_df.shape

(1732, 10)

### Number of people who made the booking in January

In [20]:
X = merged_df['no_bookings_jan'].count()
print "Number of people who made the booking in January",X

Number of people who made the booking in January 719


### Number of people who made the booking in February and also in january thus contributing to the return rate

In [21]:
Y = (merged_df['no_bookings_jan'].notnull() & merged_df['no_bookings_feb'].notnull()).sum()
print "Number of people who made the booking in Jan and Feb", Y

Number of people who made the booking in Jan and Feb 133


In [22]:
repeat_rate = (Y/float(X))*100
print "The repeat rate in percent is ",repeat_rate

The repeat rate in percent is  18.4979137691


### For each city, give the top 3 revenue earning hotels over this time period. (Not separately for Jan, Feb, Mar)

In [23]:
merged_jan_df.head()

,booking_id,customer_id,source,status,checkin,checkout,oyo_rooms,hotel_id,amount,discount,date,city,room_nights,avg_price_per_room
0,1,189314,4,3,2017-02-04,2017-02-05,1,252,3160,669.0,2017-01-31,Bangalore,1,3160.0
1,2,46268,4,3,2017-01-27,2017-01-28,1,252,1893,481.0,2017-01-27,Bangalore,1,1893.0
2,3,55271,3,2,2017-01-25,2017-01-26,1,252,2188,463.0,2017-01-25,Bangalore,1,2188.0
3,4,170766,4,3,2017-01-26,2017-01-27,1,252,3054,646.0,2017-01-26,Bangalore,1,3054.0
4,5,170766,4,3,2017-01-26,2017-01-28,1,252,6107,1293.0,2017-01-25,Bangalore,2,3053.5


In [24]:
merged_jan_df.count()

booking_id            1000
customer_id           1000
source                1000
status                1000
checkin               1000
checkout              1000
oyo_rooms             1000
hotel_id              1000
amount                1000
discount               979
date                  1000
city                  1000
room_nights           1000
avg_price_per_room    1000
dtype: int64

In [25]:
jan_hotel_df = merged_jan_df.groupby(["hotel_id","city"]).agg({"amount": ['sum']}).reset_index()
jan_hotel_df.columns = [' '.join(col).strip() for col in jan_hotel_df.columns.values]
jan_hotel_df.rename(columns={'amount sum': 'total_amt_jan'}, inplace=True)
print jan_hotel_df.shape
print jan_hotel_df.count()
jan_hotel_df.head()

(242, 3)
hotel_id         242
city             242
total_amt_jan    242
dtype: int64


,hotel_id,city,total_amt_jan
0,3,Gurgaon,118875
1,13,Gurgaon,15201
2,16,Gurgaon,14684
3,21,Gurgaon,12604
4,25,Delhi,6498


In [26]:
feb_hotel_df = merged_feb_df.groupby(["hotel_id","city"]).agg({"amount": ['sum']}).reset_index()
feb_hotel_df.columns = [' '.join(col).strip() for col in feb_hotel_df.columns.values]
feb_hotel_df.rename(columns={'amount sum': 'total_amt_feb'}, inplace=True)
print feb_hotel_df.shape
feb_hotel_df.head()

(252, 3)


,hotel_id,city,total_amt_feb
0,3,Gurgaon,9096
1,13,Gurgaon,40797
2,16,Gurgaon,16894
3,21,Gurgaon,24299
4,25,Delhi,45611


In [27]:
mar_hotel_df = merged_mar_df.groupby(["hotel_id","city"]).agg({"amount": ['sum']}).reset_index()
mar_hotel_df.columns = [' '.join(col).strip() for col in mar_hotel_df.columns.values]
mar_hotel_df.rename(columns={'amount sum': 'total_amt_mar'}, inplace=True)
print mar_hotel_df.shape
mar_hotel_df.head()

(252, 3)


,hotel_id,city,total_amt_mar
0,3,Gurgaon,4418
1,13,Gurgaon,28519
2,16,Gurgaon,2599
3,21,Gurgaon,4678
4,25,Delhi,20362


In [28]:
merged_hotel_df = pd.merge(pd.merge(jan_hotel_df,feb_hotel_df,how='left',left_on=['hotel_id','city'],
                  right_on = ['hotel_id','city']),mar_hotel_df, how='left', 
                  left_on=['hotel_id','city'],right_on = ['hotel_id','city'])
print "Shape of the merged df is - ", merged_hotel_df.shape

Shape of the merged df is -  (242, 5)


In [29]:
merged_hotel_df.head()

,hotel_id,city,total_amt_jan,total_amt_feb,total_amt_mar
0,3,Gurgaon,118875,9096.0,4418.0
1,13,Gurgaon,15201,40797.0,28519.0
2,16,Gurgaon,14684,16894.0,2599.0
3,21,Gurgaon,12604,24299.0,4678.0
4,25,Delhi,6498,45611.0,20362.0


In [30]:
merged_hotel_df.count()

hotel_id         242
city             242
total_amt_jan    242
total_amt_feb    184
total_amt_mar    165
dtype: int64

In [31]:
merged_hotel_df.fillna(value=0,inplace=True)

In [32]:
merged_hotel_df.count()

hotel_id         242
city             242
total_amt_jan    242
total_amt_feb    242
total_amt_mar    242
dtype: int64

In [33]:
merged_hotel_df['total_3_months'] = merged_hotel_df['total_amt_jan']+merged_hotel_df['total_amt_feb']\
                                    + merged_hotel_df['total_amt_mar']
merged_hotel_df.head()

,hotel_id,city,total_amt_jan,total_amt_feb,total_amt_mar,total_3_months
0,3,Gurgaon,118875,9096.0,4418.0,132389.0
1,13,Gurgaon,15201,40797.0,28519.0,84517.0
2,16,Gurgaon,14684,16894.0,2599.0,34177.0
3,21,Gurgaon,12604,24299.0,4678.0,41581.0
4,25,Delhi,6498,45611.0,20362.0,72471.0


In [34]:
merged_hotel_df.groupby('city')['total_3_months'].nlargest(3)

city          
Bangalore  22     183999.0
           163    158902.0
           27     150401.0
Chennai    191    178729.0
           237     67907.0
           86      37376.0
Delhi      97     170340.0
           93     157504.0
           89     120924.0
Gurgaon    106    177968.0
           7      151725.0
           128    136406.0
Hyderabad  193     58151.0
           194     50409.0
           240     49447.0
Jaipur     197     56537.0
           175     47357.0
           34      44580.0
Kolkata    92      29241.0
           229     12281.0
           115      9485.0
Mumbai     33     143225.0
           84     138677.0
           207    133563.0
Noida      49     222381.0
           187     59064.0
           81      57940.0
Pune       169    107255.0
           200     38925.0
           173     34631.0
Name: total_3_months, dtype: float64